# Neural Networks 101 with `gluon` - Part II

Convoultion 기반의 LeNet을 구현해본다.   

In [1]:
%pylab inline

from __future__ import print_function

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import mxnet as mx
from mxnet import nd, autograd, gluon
from mxnet.gluon import nn

# ctx = mx.cpu()
ctx = mx.gpu(0)

Populating the interactive namespace from numpy and matplotlib


## `Block` composition and LeNet

`Sequential`은 여러 블록을 하나의 블록으로 결합해주는 역할을 한다. 

`Sequential`로 간단하게 LeNet을 구현해본다. 

![](The-LeNet-5-Architecture-a-convolutional-neural-network.png.jpg)

In [2]:
lenet = nn.Sequential()

In [3]:
lenet.add(
    nn.Conv2D(channels=6, kernel_size=5, activation='relu'),
    nn.MaxPool2D(pool_size=2, strides=2),
    nn.Conv2D(channels=16, kernel_size=3, activation='relu'),
    nn.MaxPool2D(pool_size=2, strides=2),
    nn.Flatten(),
    nn.Dense(120, activation="relu"),
    nn.Dense(84, activation="relu"),
    nn.Dense(10))

In [4]:
lenet

Sequential(
  (0): Conv2D(None -> 6, kernel_size=(5, 5), stride=(1, 1))
  (1): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  (2): Conv2D(None -> 16, kernel_size=(3, 3), stride=(1, 1))
  (3): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  (4): Flatten
  (5): Dense(None -> 120, Activation(relu))
  (6): Dense(None -> 84, Activation(relu))
  (7): Dense(None -> 10, linear)
)

## `HybridBlock` (Just-in-time compilation)

1. `gluon`: 쉽게 딥러닝 구조를 프로그램 할 수 있다. 
1. 오버헤드: Python과 MXNet 백엔드간의 context switching  
1. hybridize: JIT 컴파일 

`.hybridize()` 함수를 호출함으로써 학습이 약 2배 빨라짐 

In [11]:
lenet.hybridize()

In [8]:
lenet = nn.HybridSequential()
lenet.add(
    nn.Conv2D(channels=6, kernel_size=5, activation='relu'),
    nn.MaxPool2D(pool_size=2, strides=2),
    nn.Conv2D(channels=16, kernel_size=3, activation='relu'),
    nn.MaxPool2D(pool_size=2, strides=2),
    nn.Flatten(),
    nn.Dense(120, activation="relu"),
    nn.Dense(84, activation="relu"),
    nn.Dense(10))

In [9]:
lenet.hybridize(static_alloc=True, static_shape=True)

In [10]:
lenet

HybridSequential(
  (0): Conv2D(None -> 6, kernel_size=(5, 5), stride=(1, 1))
  (1): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  (2): Conv2D(None -> 16, kernel_size=(3, 3), stride=(1, 1))
  (3): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  (4): Flatten
  (5): Dense(None -> 120, Activation(relu))
  (6): Dense(None -> 84, Activation(relu))
  (7): Dense(None -> 10, linear)
)

### Initialization

In [12]:
lenet.initialize(ctx=ctx)
# Input shape  (batch_size, color_channels, height, width)
x = nd.random.uniform(shape=(4,1,28,28), ctx=ctx)
y = lenet(x)
y.shape

(4, 10)

In [13]:
lenet

HybridSequential(
  (0): Conv2D(1 -> 6, kernel_size=(5, 5), stride=(1, 1))
  (1): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  (2): Conv2D(6 -> 16, kernel_size=(3, 3), stride=(1, 1))
  (3): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  (4): Flatten
  (5): Dense(400 -> 120, Activation(relu))
  (6): Dense(120 -> 84, Activation(relu))
  (7): Dense(84 -> 10, linear)
)

### MNIST

In [14]:
def normalize_and_copy(dataset, ctx):
    data = mx.nd.array(dataset._data, ctx=ctx) \
                .transpose((0, 3, 1, 2)) \
                .astype(np.float32)/255
    label = dataset._label
    output = gluon.data.ArrayDataset(data, label)
    output._data[1] = mx.nd.array(label, ctx=ctx)
    return output

In [15]:
train_data = normalize_and_copy(gluon.data.vision.MNIST(train=True), ctx)
test_data = normalize_and_copy(gluon.data.vision.MNIST(train=False), ctx)

In [16]:
batch_size = 64

train_batches = mx.gluon.data.DataLoader(train_data, batch_size, shuffle=True)
test_batches = mx.gluon.data.DataLoader(test_data, batch_size, shuffle=False)

In [17]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(lenet.collect_params(), 'sgd', {'learning_rate': .02})

### Evaluation

In [18]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    for i, (data, label) in enumerate(data_iterator):
        prediction = net(data).argmax(axis=-1, keepdims=True)
        acc.update(preds=prediction, labels=label)
    return acc.get()[1]

### Training

In [19]:
epochs = 10

In [20]:
for e in range(epochs):
    cumulative_loss = 0
    for i, (data, label) in enumerate(train_batches):
        with autograd.record():
            output = lenet(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(data.shape[0])
        cumulative_loss += nd.sum(loss)

    train_accuracy = evaluate_accuracy(train_batches, lenet)
    test_accuracy = evaluate_accuracy(test_batches, lenet)
    print("Epoch %s. Loss: %.4f, Train_acc %.4f, Test_acc %.4f" %
          (e, cumulative_loss.asscalar()/len(train_data), train_accuracy, test_accuracy))

Epoch 0. Loss: 2.2992, Train_acc 0.1201, Test_acc 0.1224
Epoch 1. Loss: 1.3274, Train_acc 0.8492, Test_acc 0.8580
Epoch 2. Loss: 0.3183, Train_acc 0.9251, Test_acc 0.9299
Epoch 3. Loss: 0.1916, Train_acc 0.9500, Test_acc 0.9510
Epoch 4. Loss: 0.1378, Train_acc 0.9633, Test_acc 0.9659
Epoch 5. Loss: 0.1102, Train_acc 0.9633, Test_acc 0.9631
Epoch 6. Loss: 0.0942, Train_acc 0.9717, Test_acc 0.9703
Epoch 7. Loss: 0.0830, Train_acc 0.9781, Test_acc 0.9773
Epoch 8. Loss: 0.0743, Train_acc 0.9713, Test_acc 0.9703
Epoch 9. Loss: 0.0667, Train_acc 0.9758, Test_acc 0.9747
